Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

Tune L2 regularization for logistic model

In [5]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # L2 regularization for the fully connected parameters.
  regularizers = tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases) 
  # Add the regularization term to the loss.
  loss += 5e-4 * regularizers
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 20.073441
Minibatch accuracy: 11.7%
Validation accuracy: 12.5%
Minibatch loss at step 500: 2.079197
Minibatch accuracy: 77.3%
Validation accuracy: 75.8%
Minibatch loss at step 1000: 1.744632
Minibatch accuracy: 80.5%
Validation accuracy: 77.4%
Minibatch loss at step 1500: 1.098860
Minibatch accuracy: 84.4%
Validation accuracy: 78.4%
Minibatch loss at step 2000: 1.023061
Minibatch accuracy: 80.5%
Validation accuracy: 79.2%
Minibatch loss at step 2500: 1.092160
Minibatch accuracy: 76.6%
Validation accuracy: 80.4%
Minibatch loss at step 3000: 0.897657
Minibatch accuracy: 82.0%
Validation accuracy: 80.9%
Test accuracy: 88.4%


Initialized
Minibatch loss at step 0: 16.625013
Minibatch accuracy: 15.6%
Validation accuracy: 18.1%
Minibatch loss at step 500: 1.974661
Minibatch accuracy: 82.0%
Validation accuracy: 76.0%
Minibatch loss at step 1000: 1.791342
Minibatch accuracy: 79.7%
Validation accuracy: 77.5%
Minibatch loss at step 1500: 1.106296
Minibatch accuracy: 83.6%
Validation accuracy: 78.5%
Minibatch loss at step 2000: 0.973265
Minibatch accuracy: 86.7%
Validation accuracy: 79.3%
Minibatch loss at step 2500: 0.962462
Minibatch accuracy: 78.1%
Validation accuracy: 80.5%
Minibatch loss at step 3000: 0.943414
Minibatch accuracy: 79.7%
Validation accuracy: 81.0%
Test accuracy: 88.7%

Tune L2 regularization for neural network model

In [7]:
batch_size = 128
hidden_layer_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weightsHidden = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biasesHidden = tf.Variable(tf.zeros([hidden_layer_size]))
  
  logitsHidden = tf.matmul(tf_train_dataset, weightsHidden) + biasesHidden
  hiddenLayer = tf.nn.relu(logitsHidden)

  weights = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = tf.matmul(hiddenLayer, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # L2 regularization for the fully connected parameters.
  regularizers = (tf.nn.l2_loss(weightsHidden) + tf.nn.l2_loss(biasesHidden) +
                  tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
  # Add the regularization term to the loss.
  loss += 5e-4 * regularizers
 
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
    
  valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, weightsHidden) + biasesHidden)
  valid_logits = tf.matmul(valid_hidden, weights) + biases
  valid_prediction = tf.nn.softmax(valid_logits)
    
  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, weightsHidden) + biasesHidden)
  test_logits = tf.matmul(test_hidden, weights) + biases
  test_prediction = tf.nn.softmax(test_logits)

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 471.523987
Minibatch accuracy: 14.8%
Validation accuracy: 21.1%
Minibatch loss at step 500: 134.516937
Minibatch accuracy: 81.2%
Validation accuracy: 79.2%
Minibatch loss at step 1000: 97.997849
Minibatch accuracy: 78.9%
Validation accuracy: 81.8%
Minibatch loss at step 1500: 74.139778
Minibatch accuracy: 85.2%
Validation accuracy: 82.2%
Minibatch loss at step 2000: 57.005882
Minibatch accuracy: 89.8%
Validation accuracy: 82.6%
Minibatch loss at step 2500: 44.487976
Minibatch accuracy: 85.2%
Validation accuracy: 83.9%
Minibatch loss at step 3000: 34.578785
Minibatch accuracy: 85.2%
Validation accuracy: 84.2%
Test accuracy: 91.3%


Initialized
Minibatch loss at step 0: 520.794128
Minibatch accuracy: 12.5%
Validation accuracy: 27.2%
Minibatch loss at step 500: 138.610382
Minibatch accuracy: 80.5%
Validation accuracy: 79.5%
Minibatch loss at step 1000: 99.893425
Minibatch accuracy: 78.1%
Validation accuracy: 80.6%
Minibatch loss at step 1500: 74.578651
Minibatch accuracy: 86.7%
Validation accuracy: 80.0%
Minibatch loss at step 2000: 57.215748
Minibatch accuracy: 89.1%
Validation accuracy: 83.1%
Minibatch loss at step 2500: 44.275105
Minibatch accuracy: 86.7%
Validation accuracy: 83.4%
Minibatch loss at step 3000: 34.290161
Minibatch accuracy: 88.3%
Validation accuracy: 84.2%
Test accuracy: 91.4%

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [9]:
batch_size = 12

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # L2 regularization for the fully connected parameters.
  regularizers = tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases) 
  # Add the regularization term to the loss.
  loss += 5e-4 * regularizers
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [10]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 19.684132
Minibatch accuracy: 16.7%
Validation accuracy: 12.9%
Minibatch loss at step 500: 1.609565
Minibatch accuracy: 83.3%
Validation accuracy: 73.5%
Minibatch loss at step 1000: 1.243329
Minibatch accuracy: 83.3%
Validation accuracy: 72.4%
Minibatch loss at step 1500: 0.944184
Minibatch accuracy: 91.7%
Validation accuracy: 75.2%
Minibatch loss at step 2000: 4.144327
Minibatch accuracy: 66.7%
Validation accuracy: 73.1%
Minibatch loss at step 2500: 2.368257
Minibatch accuracy: 66.7%
Validation accuracy: 73.4%
Minibatch loss at step 3000: 1.800102
Minibatch accuracy: 83.3%
Validation accuracy: 75.9%
Test accuracy: 82.7%


In [11]:
batch_size = 12
hidden_layer_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weightsHidden = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biasesHidden = tf.Variable(tf.zeros([hidden_layer_size]))
  
  logitsHidden = tf.matmul(tf_train_dataset, weightsHidden) + biasesHidden
  hiddenLayer = tf.nn.relu(logitsHidden)

  weights = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = tf.matmul(hiddenLayer, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # L2 regularization for the fully connected parameters.
  regularizers = (tf.nn.l2_loss(weightsHidden) + tf.nn.l2_loss(biasesHidden) +
                  tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
  # Add the regularization term to the loss.
  loss += 5e-4 * regularizers
 
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
    
  valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, weightsHidden) + biasesHidden)
  valid_logits = tf.matmul(valid_hidden, weights) + biases
  valid_prediction = tf.nn.softmax(valid_logits)
    
  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, weightsHidden) + biasesHidden)
  test_logits = tf.matmul(test_hidden, weights) + biases
  test_prediction = tf.nn.softmax(test_logits)

In [12]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 479.397095
Minibatch accuracy: 25.0%
Validation accuracy: 17.1%
Minibatch loss at step 500: 758.940125
Minibatch accuracy: 75.0%
Validation accuracy: 60.5%
Minibatch loss at step 1000: 488.981354
Minibatch accuracy: 50.0%
Validation accuracy: 52.9%
Minibatch loss at step 1500: 600.512329
Minibatch accuracy: 58.3%
Validation accuracy: 54.7%
Minibatch loss at step 2000: 584.767761
Minibatch accuracy: 75.0%
Validation accuracy: 53.6%
Minibatch loss at step 2500: 429.065979
Minibatch accuracy: 16.7%
Validation accuracy: 55.5%
Minibatch loss at step 3000: 340.685486
Minibatch accuracy: 66.7%
Validation accuracy: 60.0%
Test accuracy: 66.0%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [22]:
batch_size = 50
hidden_layer_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weightsHidden = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biasesHidden = tf.Variable(tf.zeros([hidden_layer_size]))
  
  logitsHidden = tf.matmul(tf_train_dataset, weightsHidden) + biasesHidden
  hiddenLayer = tf.nn.relu(logitsHidden)



  # Add a 50% dropout during training only. Dropout also scales
  # activations such that no rescaling is needed at evaluation time.
  hidden = tf.nn.dropout(hiddenLayer, 0.5, seed=None)

  weights = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))  
  
  # Training computation.
  logits = tf.matmul(hidden, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # L2 regularization for the fully connected parameters.
  regularizers = (tf.nn.l2_loss(weightsHidden) + tf.nn.l2_loss(biasesHidden) +
                  tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
  # Add the regularization term to the loss.
  loss += 5e-4 * regularizers
 
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
    
  valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, weightsHidden) + biasesHidden)
  valid_logits = tf.matmul(valid_hidden, weights) + biases
  valid_prediction = tf.nn.softmax(valid_logits)
    
  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, weightsHidden) + biasesHidden)
  test_logits = tf.matmul(test_hidden, weights) + biases
  test_prediction = tf.nn.softmax(test_logits)

In [ ]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 569.167786
Minibatch accuracy: 8.0%
Validation accuracy: 23.2%
Minibatch loss at step 500: 252.663818
Minibatch accuracy: 68.0%
Validation accuracy: 72.8%
Minibatch loss at step 1000: 302.984375
Minibatch accuracy: 64.0%
Validation accuracy: 74.8%
Minibatch loss at step 1500: 236.571564
Minibatch accuracy: 68.0%
Validation accuracy: 74.4%
Minibatch loss at step 2000: 180.924744
Minibatch accuracy: 56.0%
Validation accuracy: 71.8%
Minibatch loss at step 2500: 112.501534
Minibatch accuracy: 72.0%
Validation accuracy: 70.3%

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
